# Blood transfusion service center

V1: Recency - months since last donation <br>
V2: Frequency - total number of donation <br>
V3: Monetary - total blood donated in c.c. <br>
V4: Time - months since first donation), and a binary variable representing whether he/she donated blood in  March 2007 (1 stand for donating blood; 0 stands for not donating blood).

The target attribute is a binary variable representing whether <br>
he/she donated blood in March 2007 (2 stands for donating blood; 1 stands for not donating blood).

### The dataset is taken from https://www.openml.org/d/1464

In [32]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.utils import np_utils
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle
from sklearn.utils import resample

from google.colab import files
uploaded = files.upload()
import io

Saving blood-transfusion-service-center.csv to blood-transfusion-service-center (1).csv


In [9]:

df = pd.read_csv(io.BytesIO(uploaded['blood-transfusion-service-center.csv']))
df.head()

,V1,V2,V3,V4,Class
0,2,50,12500,98,2
1,0,13,3250,28,2
2,1,16,4000,35,2
3,2,20,5000,45,2
4,1,24,6000,77,1


In [17]:
!pip install sweetviz

     |████████████████████████████████| 15.1MB 305kB/s 
     |████████████████████████████████| 81kB 6.1MB/s 
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [27]:
import pandas as pd
import sweetviz as sv

#EDA using Autoviz
sweet_report = sv.analyze(pd.read_csv("blood-transfusion-service-center.csv",sep='delimiter', header=None))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """


                                             |          | [  0%]   00:00 -> (? left)

In [28]:
#Saving results to HTML file
sweet_report.show_html('blood-transfusion-service-center.csv')

Report blood-transfusion-service-center.csv was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [29]:
df.shape

(748, 5)

In [30]:
df = df.dropna(how='all')

In [31]:
df["Class"].value_counts()

1    570
2    178
Name: Class, dtype: int64

In [33]:
df_majority = df[df.Class==2]
df_minority = df[df.Class==1]

# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=178,    # to match majority class
                                 random_state=42) # reproducible results


In [34]:

#Combine majority class with upsampled minority class
df = pd.concat([df_majority, df_minority_upsampled])

#Display new class counts
df.Class.value_counts()


2    178
1    178
Name: Class, dtype: int64

In [36]:
df["Class"].value_counts()

2    178
1    178
Name: Class, dtype: int64

In [37]:
X = df.drop(['Class'], axis=1).values
#X = StandardScaler().fit_transform(X)
Y = df['Class']

In [38]:
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size = 0.30, random_state = 101)

In [39]:
trainedforest = RandomForestClassifier(n_estimators=700).fit(X_Train,Y_Train)
predictionforest = trainedforest.predict(X_Test)
trainedforest.score(X_Train, Y_Train)


0.9116465863453815

In [40]:
trainedforest = RandomForestClassifier(n_estimators=700).fit(X,Y)

In [41]:
# Saving model to disk
pickle.dump(trainedforest, open('model.pkl','wb'))

In [42]:
# Loading model to compare the results
model = pickle.load(open('model.pkl','rb'))

In [43]:
print(model.predict([[2,  430, 10350,  86]]))

[2]


In [44]:
p = model.predict(X_Test)
print(X_Test)
print(list(p).count(1))
print(list(p).count(2))

[[   11     2   500    11]
 [    2     1   250     2]
 [    0     3   750     4]
 [    2    43 10750    86]
 [    2    11  2750    26]
 [   23     1   250    23]
 [   11     2   500    11]
 [    2     2   500    33]
 [    4     5  1250    46]
 [   16     4  1000    33]
 [   14     1   250    14]
 [   11     9  2250    72]
 [    2     1   250     2]
 [    4     2   500     4]
 [   23     6  1500    70]
 [   38     1   250    38]
 [    4     8  2000    28]
 [    8     8  2000    52]
 [    4     4  1000     4]
 [   16     5  1250    40]
 [   11     5  1250    33]
 [   14     5  1250    74]
 [    4     1   250     4]
 [   21     2   500    33]
 [    2     6  1500    15]
 [   14    16  4000    98]
 [    4     3   750    16]
 [    4     5  1250    11]
 [    4     9  2250    38]
 [    9     1   250     9]
 [    2     3   750     4]
 [    2     6  1500    16]
 [    2     8  2000    38]
 [    4     4  1000     4]
 [    8    15  3750    77]
 [   14     2   500    14]
 [    4    12  3000    34]
 

### The EDA was done using the sweetviz lib and the result is stored in EDA.html file. The model was created using python pandas and sklearn libraries and was then stored to be ready to make predictions using pickle Which will be implemented in future.  